In [ ]:
import os
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# Get current directory
CURR_DIR = os.getcwd()

# Get MNIST 
# One-hot encoding is used for categorical variables, as binary vector
mnist = input_data.read_data_sets(CURR_DIR + "/data/", one_hot=True)

In [ ]:
# Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

In [ ]:
# Network Parameters
hidden_layer_1 = 256
hidden_layer_2 = 256
num_input = 784
num_classes = 10

In [ ]:
# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [ ]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, hidden_layer_1])),
    'h2': tf.Variable(tf.random_normal([hidden_layer_1, hidden_layer_2])),
    'out': tf.Variable(tf.random_normal([hidden_layer_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([hidden_layer_1])),
    'b2': tf.Variable(tf.random_normal([hidden_layer_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [ ]:
# Create Neural Network Model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    
    return out_layer

In [ ]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))